In [ ]:
!pip install -q ultralytics kagglehub

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hasnain1234","key":"60e1947e8eacbc53ddb7727533744614"}'}

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshaymiddha/crack-segmentation-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/crack-segmentation-dataset


In [ ]:
!ls /kaggle/input/crack-segmentation-dataset/test

ls: cannot access '/kaggle/input/crack-segmentation-dataset/test': No such file or directory


In [ ]:
!ls /root/.cache/kagglehub/datasets/lakshaymiddha/crack-segmentation-dataset/versions/1/crack_segmentation_dataset/train

images	labels	labels.cache  masks


In [ ]:
import os

# Dataset path (change to "train" if needed)
dataset_path = "/root/.cache/kagglehub/datasets/lakshaymiddha/crack-segmentation-dataset/versions/1/crack_segmentation_dataset/train/images"

# List all files containing 'noncrack' in their name
noncrack_images = [
    f for f in os.listdir(dataset_path)
    if "noncrack" in f.lower() and f.lower().endswith((".jpg", ".png", ".jpeg"))
]

# Print results
print(f"Total noncrack images in test folder: {len(noncrack_images)}")
print("Sample noncrack image filenames:")
print(noncrack_images[:10])


Total noncrack images in test folder: 1199
Sample noncrack image filenames:
['noncrack_noncrack_concrete_wall_85_0.jpg.jpg', 'noncrack_noncrack_concrete_wall_77_4.jpg.jpg', 'noncrack_noncrack_concrete_wall_60_14.jpg.jpg', 'noncrack_noncrack_concrete_wall_74_29.jpg.jpg', 'noncrack_noncrack_concrete_wall_16_6.jpg.jpg', 'noncrack_noncrack_concrete_wall_24_21.jpg.jpg', 'noncrack_noncrack_concrete_wall_45_6.jpg.jpg', 'noncrack_noncrack_concrete_wall_9_15.jpg.jpg', 'noncrack_noncrack_concrete_wall_16_0.jpg.jpg', 'noncrack_noncrack_concrete_wall_60_16.jpg.jpg']


In [ ]:
import os
import cv2
import numpy as np

def convert_masks_to_yolo_seg(image_dir, mask_dir, label_dir):
    """
    Convert binary mask images to YOLO segmentation format.

    Args:
        image_dir: Directory containing original images
        mask_dir: Directory containing binary mask images (white=crack, black=background)
        label_dir: Output directory for YOLO format label files
    """
    os.makedirs(label_dir, exist_ok=True)

    # Get all jpg files from mask directory
    files = [f for f in os.listdir(mask_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    processed = 0
    empty_masks = 0

    for filename in files:
        mask_path = os.path.join(mask_dir, filename)
        image_path = os.path.join(image_dir, filename)
        label_path = os.path.join(label_dir, filename.rsplit('.', 1)[0] + '.txt')

        # Read mask image
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print(f"Warning: Could not read {mask_path}")
            continue

        height, width = mask.shape

        # Create binary mask (white pixels = 255 become crack regions)
        _, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

        # Check if mask is empty (no cracks detected)
        if cv2.countNonZero(binary_mask) == 0:
            # Create empty label file for images without cracks
            with open(label_path, 'w') as f:
                pass  # Empty file
            empty_masks += 1
            continue

        # Find contours of crack regions
        contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        with open(label_path, "w") as f:
            for contour in contours:
                # Simplify contour to reduce points while preserving shape
                epsilon = 0.002 * cv2.arcLength(contour, True)  # Adjust epsilon for more/less simplification
                simplified_contour = cv2.approxPolyDP(contour, epsilon, True)

                # Skip very small contours (likely noise)
                if len(simplified_contour) < 3:
                    continue

                # Flatten the contour array
                points = simplified_contour.reshape(-1, 2)

                # Skip if contour is too small
                if len(points) < 3:
                    continue

                # Normalize coordinates to [0, 1] range
                normalized_points = []
                for x, y in points:
                    norm_x = max(0, min(1, x / width))   # Clamp to [0, 1]
                    norm_y = max(0, min(1, y / height))  # Clamp to [0, 1]
                    normalized_points.extend([norm_x, norm_y])

                # Write YOLO segmentation format: class_id x1 y1 x2 y2 x3 y3 ...
                # Class 0 for crack
                coords_str = " ".join(f"{coord:.6f}" for coord in normalized_points)
                f.write(f"0 {coords_str}\n")

        processed += 1
        if processed % 100 == 0:
            print(f"Processed {processed} files...")

    print(f"Processed {processed} files, {empty_masks} had no cracks")

def main():
    # Dataset paths
    base_path = "/root/.cache/kagglehub/datasets/lakshaymiddha/crack-segmentation-dataset/versions/1/crack_segmentation_dataset"
    splits = ["train", "test"]

    for split in splits:
        print(f"\nProcessing {split} split...")
        image_dir = os.path.join(base_path, split, "images")
        mask_dir = os.path.join(base_path, split, "masks")
        label_dir = os.path.join(base_path, split, "labels")

        # Check if directories exist
        if not os.path.exists(image_dir):
            print(f"Warning: Image directory not found: {image_dir}")
            continue
        if not os.path.exists(mask_dir):
            print(f"Warning: Mask directory not found: {mask_dir}")
            continue

        convert_masks_to_yolo_seg(image_dir, mask_dir, label_dir)

    print("\n✅ YOLO segmentation label generation completed!")

main()


Processing train split...
Processed 100 files...
Processed 200 files...
Processed 300 files...
Processed 400 files...
Processed 500 files...
Processed 600 files...
Processed 700 files...
Processed 800 files...
Processed 900 files...
Processed 1000 files...
Processed 1100 files...
Processed 1200 files...
Processed 1300 files...
Processed 1400 files...
Processed 1500 files...
Processed 1600 files...
Processed 1700 files...
Processed 1800 files...
Processed 1900 files...
Processed 2000 files...
Processed 2100 files...
Processed 2200 files...
Processed 2300 files...
Processed 2400 files...
Processed 2500 files...
Processed 2600 files...
Processed 2700 files...
Processed 2800 files...
Processed 2900 files...
Processed 3000 files...
Processed 3100 files...
Processed 3200 files...
Processed 3300 files...
Processed 3400 files...
Processed 3500 files...
Processed 3600 files...
Processed 3700 files...
Processed 3800 files...
Processed 3900 files...
Processed 4000 files...
Processed 4100 files..

In [ ]:
# =============================================================================
# AGGRESSIVE COLAB CRASH PREVENTION
# =============================================================================
import gc
import torch
import psutil
import os

# Force garbage collection multiple times
for _ in range(3):
    gc.collect()

# Clear all CUDA memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()  # Clear IPC memory
    print("✅ GPU memory cleared")
    print(f"GPU Memory available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Check memory
print(f"Current RAM usage: {psutil.virtual_memory().percent:.1f}%")
print(f"Available RAM: {psutil.virtual_memory().available / 1e9:.1f} GB")

# Set environment variables for memory optimization
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

from ultralytics import YOLO

# ULTRA-CONSERVATIVE SETTINGS FOR COLAB
model = YOLO("yolov8n-seg.pt")

model.train(
    data="crackSegment.yaml",
    epochs=16,                      # REDUCED epochs to prevent timeout
    imgsz=256,                      # SMALLER image size (was 320)
    batch=8,                       # SMALLER batch
    cache=False,                    # NO image caching
    device='cuda',
    plots=False,
    name="segmentCracksYOLOv8n-seg",

    # Memory-efficient settings
    amp=True,                       # Mixed precision
    patience=4,                     # Early stopping

    # Other optimizations
    verbose=False,                  # Less printing
    deterministic=False,           # Faster training
    single_cls=True                # Single class optimization
)

print("✅ Training completed successfully!")

# Final cleanup
torch.cuda.empty_cache()
gc.collect()


✅ GPU memory cleared
GPU Memory available: 15.8 GB
Current RAM usage: 24.5%
Available RAM: 10.3 GB
Ultralytics 8.3.154 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=crackSegment.yaml, degrees=0.0, deterministic=False, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=16, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=segmentCracksYOLOv8n-seg7, nbs=64, nms=False, opset=None, optimi

train: Scanning /root/.cache/kagglehub/datasets/lakshaymiddha/crack-segmentation-dataset/versions/1/crack_segmentation_dataset/train/labels.cache... 9603 images, 10836 backgrounds, 0 corrupt: 100%|██████████| 19206/19206 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 616.8±763.9 MB/s, size: 38.1 KB)


val: Scanning /root/.cache/kagglehub/datasets/lakshaymiddha/crack-segmentation-dataset/versions/1/crack_segmentation_dataset/test/labels.cache... 1695 images, 1916 backgrounds, 0 corrupt: 100%|██████████| 3390/3390 [00:00<?, ?it/s]


...

       2/16     0.695G      1.785       2.63      1.903       1.47         16        256: 100%|██████████| 2401/2401 [07:53<00:00,  5.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 212/212 [00:26<00:00,  8.09it/s]


                   all       3390      32088      0.182     0.0438      0.027     0.0129      0.125     0.0327     0.0156    0.00362

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       3/16     0.695G      1.713      2.574      1.815      1.439         10        256: 100%|██████████| 2401/2401 [07:51<00:00,  5.09it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 212/212 [00:25<00:00,  8.19it/s]


                   all       3390      32088      0.198     0.0433     0.0291     0.0155      0.151     0.0317     0.0188    0.00431

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       4/16     0.697G      1.672      2.547      1.762      1.411         49        256:  77%|███████▋  | 1844/2401 [06:05<01:35,  5.83it/s]